# Silver Layer

In [0]:
%skip
bronze_dim_categoria_produto = spark.read.format("delta").load("/Volumes/data_ex/bronze/download001/bronze_dim_categoria_produto")
display(bronze_dim_categoria_produto)

bronze_dim_cliente = spark.read.format("delta").load("/Volumes/data_ex/bronze/download001/bronze_dim_cliente")
display(bronze_dim_cliente)

bronze_dim_data = spark.read.format("delta").load("/Volumes/data_ex/bronze/download001/bronze_dim_data")
display(bronze_dim_data)

bronze_dim_localidade = spark.read.format("delta").load("/Volumes/data_ex/bronze/download001/bronze_dim_localidade")
display(bronze_dim_localidade)

bronze_dim_produto = spark.read.format("delta").load("/Volumes/data_ex/bronze/download001/bronze_dim_produto")
display(bronze_dim_produto)

bronze_fato_vendas = spark.read.format("delta").load("/Volumes/data_ex/bronze/download001/bronze_fato_vendas")
display(bronze_fato_vendas)

## Configuração do servidor para trabalho no Silver Layer

In [0]:
# Importação das bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Iniciar a SparkSession com configurações otimizadas
spark = SparkSession.builder \
    .appName("Transform Data Silver") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Conta quantos volumes já existem dentro do schema
numero = spark.sql("show volumes in data_ex.lhdw").count()
# Armazena o nome do último volume salvo
volume_salvo = f"download{numero:03d}"

# Definição dos path bronze e silver
bronze_path = f"/Volumes/data_ex/bronze/{volume_salvo}"
silver_path = f"/Volumes/data_ex/silver/{volume_salvo}"

spark.sql(f"create volume if not exists data_ex.silver.{volume_salvo}")

## Trabalhos no Silver Layer

**Lendo a camada bronze**

In [0]:
# Importação das bibliotecas necessárias
from pyspark.sql.functions import format_number

# Lendo tabelas bronze
bronze_dim_categoria = spark.read.format("delta") \
    .load(f"{bronze_path}/bronze_dim_categoria_produto")
bronze_dim_cliente = spark.read.format("delta") \
    .load(f"{bronze_path}/bronze_dim_cliente")
bronze_dim_data = spark.read.format("delta") \
    .load(f"{bronze_path}/bronze_dim_data")
bronze_dim_localidade = spark.read.format("delta") \
    .load(f"{bronze_path}/bronze_dim_localidade")
bronze_dim_produto = spark.read.format("delta") \
    .load(f"{bronze_path}/bronze_dim_produto")
bronze_fato_vendas = spark.read.format("delta") \
    .load(f"{bronze_path}/bronze_fato_vendas")

**Tratando os dados**

In [0]:
# Função que substitui valores null por N/A
def null_to_na(df):
    for campo, tipo in df.dtypes:
        if tipo == "string" and not campo.endswith("_id"):
            df = df.withColumn(campo, when(col(campo).isNull(), "N/A").otherwise(col(campo)))
    return df

# Definindo tabelas silver
# Retirando duplicatas, padronizando nomes de colunas, substituindo valores null
silver_dim_categoria = bronze_dim_categoria \
.dropDuplicates(["categoria_id"]) \
.withColumn("categoria_nome", upper(trim(col("categoria_nome"))))
silver_dim_categoria = null_to_na(silver_dim_categoria)

silver_dim_cliente = bronze_dim_cliente \
.dropDuplicates(["cliente_id"]) \
.withColumn("nome_cliente", initcap(trim(col("nome_cliente")))) \
.withColumn("estado", upper(col("estado"))) \
.withColumn("cidade", initcap(col("cidade")))
silver_dim_cliente = null_to_na(silver_dim_cliente)

silver_dim_data = bronze_dim_data.dropDuplicates(["data_id"])
silver_dim_data = null_to_na(silver_dim_data)

silver_dim_localidade = bronze_dim_localidade \
.dropDuplicates(["localidade_id"]) \
.withColumn("estado", upper(col("estado"))) \
.withColumn("cidade", initcap(col("cidade")))
silver_dim_localidade = null_to_na(silver_dim_localidade)

silver_dim_produto = bronze_dim_produto \
.dropDuplicates(["produto_id"]) \
.withColumn("categoria_nome", upper(col("categoria_nome")))
silver_dim_produto = null_to_na(silver_dim_produto)

silver_fato_vendas = bronze_fato_vendas \
.dropDuplicates(["venda_id"]) \
.filter(col("quantidade") > 0)

## Salvar arquivos na camada Silver

**Função de Save**

In [0]:
silver_dim_categoria.write.format("delta").mode("overwrite") \
    .save(f"{silver_path}/silver_dim_categoria_produto")

silver_dim_cliente.write.format("delta").mode("overwrite") \
    .save(f"{silver_path}/silver_dim_cliente")

silver_dim_data.write.format("delta").mode("overwrite") \
    .save(f"{silver_path}/silver_dim_data")

silver_dim_localidade.write.format("delta").mode("overwrite") \
    .save(f"{silver_path}/silver_dim_localidade")

silver_dim_produto.write.format("delta").mode("overwrite") \
    .save(f"{silver_path}/silver_dim_produto")

silver_fato_vendas.write.format("delta").mode("overwrite") \
    .save(f"{silver_path}/silver_fato_vendas")

## Limpar memória

In [0]:
import gc
gc.collect()

**Teste**

In [0]:
silver_dim_categoria_produto = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_categoria_produto")
display(silver_dim_categoria_produto)

silver_dim_cliente = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_cliente")
display(silver_dim_cliente)

silver_dim_data = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_data")
display(silver_dim_data)

silver_dim_localidade = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_localidade")
display(silver_dim_localidade)

silver_dim_produto = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_dim_produto")
display(silver_dim_produto)

silver_fato_vendas = spark.read.format("delta").load("/Volumes/data_ex/silver/download001/silver_fato_vendas")
display(silver_fato_vendas)